Basic AE

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [0]:
# Importing the dataset
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
#users = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [0]:
# Preparing the training set and the test set
from sklearn.model_selection import train_test_split
training_set , test_set = train_test_split(ratings, test_size = 0.2)

training_set = np.array(training_set, dtype = 'int')

test_set = np.array(test_set, dtype = 'int')

In [0]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))


In [0]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [0]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [0]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 30)
        self.fc2 = nn.Linear(30, 20)
        self.fc3 = nn.Linear(20, 10)
        self.fc4 = nn.Linear(10, 20)
        self.fc5 = nn.Linear(20, 30)
        self.fc6 = nn.Linear(30, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.activation(self.fc4(x))
        x = self.activation(self.fc5(x))
        x = self.fc6(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)


In [0]:
nb_epoch = 50
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.item()*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 0.9959933474270751
epoch: 2 loss: 0.98965164306709
epoch: 3 loss: 0.9872563798543212
epoch: 4 loss: 0.9858974341899084
epoch: 5 loss: 0.9850043413156993
epoch: 6 loss: 0.9845463555385838
epoch: 7 loss: 0.9842012932911968
epoch: 8 loss: 0.9839147239238635
epoch: 9 loss: 0.9838595590268009
epoch: 10 loss: 0.9834634240663511
epoch: 11 loss: 0.9832962830809723
epoch: 12 loss: 0.9829701745367839
epoch: 13 loss: 0.983075605534998
epoch: 14 loss: 0.9825664988921557
epoch: 15 loss: 0.9823535087298756
epoch: 16 loss: 0.9820309834907472
epoch: 17 loss: 0.9819393313356538
epoch: 18 loss: 0.9820836426676318
epoch: 19 loss: 0.9817929843334916
epoch: 20 loss: 0.9814281013850278
epoch: 21 loss: 0.9812334472848276
epoch: 22 loss: 0.9808308527449768
epoch: 23 loss: 0.9806472292754111
epoch: 24 loss: 0.9804091248323904
epoch: 25 loss: 0.9801658133532114
epoch: 26 loss: 0.9799505977595393
epoch: 27 loss: 0.9798939486382579
epoch: 28 loss: 0.9798945496707188
epoch: 29 loss: 0.97959183715044

In [0]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 0.9858143215597271
